## Import libraries

In [ ]:
!pip install langchain

In [68]:
!pip install -U pypdf
!pip install -U pypdf2
!pip install -U langchain
!pip install -U langchain-experimental
!pip install -U langchain-community
!pip install -U langchain-pinecone
!pip install -U sentence-transformers
!pip install -U pinecone-client
!pip install -U openai
!pip install -U pandas
!pip install -U numpy
!pip install -U scikit-learn
!pip install -U matplotlib
!pip install -U seaborn
!pip install -U gunicorn
!pip install -U python-dotenv
!pip install -U flask-cors
!pip install -U requests
!pip install -U openai
!pip install -U python-dotenv
!pip install -U flask-cors
!pip install -U requests

from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


  Using cached langchain-0.3.14-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_core-0.3.29-py3-none-any.whl.metadata (6.3 kB)
Using cached langchain-0.3.14-py3-none-any.whl (1.0 MB)
Using cached langchain_core-0.3.29-py3-none-any.whl (411 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.28
    Uninstalling langchain-core-0.3.28:
      Successfully uninstalled langchain-core-0.3.28
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.13
    Uninstalling langchain-0.3.13:
      Successfully uninstalled langchain-0.3.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-pinecone 0.0.1 requires pinecone-client<4,>=3; python_version >= "3.8" and python_version < "3.13", but you have pinecone-client 5.0.1 which is incompatible.
  Using cached langchain_community-0.3.14-py3-n

## Let extract the data from the pdf file using fucntions

In [66]:

def load_data(data):
    loader = DirectoryLoader(data,
                            glob='*.pdf',
                            loader_cls=PyPDFLoader)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    docs = text_splitter.split_documents(documents)
    return docs

# Extract the content from the pdf file

In [69]:


extracted_data = load_data(data='/Users/kwamebaffoe/Desktop/End-End-Generative-AI-Medical-Chatbot/Data')

In [70]:
# extracted_data

# Let Split the data into chunks using fucntions
def split_data(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks


In [73]:
text_chunks = split_data(extracted_data)
print(len(text_chunks))

4109


# Let download the embeddings from Huggingface

In [74]:

from langchain.embeddings import HuggingFaceEmbeddings

In [75]:
from langchain.embeddings import HuggingFaceEmbeddings  

def download_huggingface_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [76]:
# !pip install sentence-transformers
embeddings = download_huggingface_embeddings()


#  Let's convert to Vector Embeddings 

In [77]:
from langchain.embeddings import HuggingFaceEmbeddings  # Ensure you import this

def download_huggingface_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

# Call the function to get the embeddings
embeddings = download_huggingface_embeddings()

In [78]:
# TEST CODES

from sentence_transformers import SentenceTransformer
from huggingface_hub import cached_download

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
print("Model loaded successfully!")


ImportError: cannot import name 'cached_download' from 'huggingface_hub' (/opt/anaconda3/envs/einstienMed/lib/python3.10/site-packages/huggingface_hub/__init__.py)

In [79]:
# Now you can use the embeddings to embed the query
# !pip install sentence-transformers
import sentence_transformers

query_output = embeddings.embed_query("What is a heart attack?")
print("Length of query output: ", len(query_output))

Length of query output:  384


In [ ]:
# query_output

# !pip install google
# !pip install "sentence-transformers>=3.0.0"
# !pip install sentence-transformers
# !pip install "torch>=1.11.0"

# !pip install torch==1.11.0
# !pip install sentence-transformers

# Example: Install torch with a specified CUDA or CPU version
# !pip install torch==1.11.0+cpu -f https://download.pytorch.org/whl/cpu/torch_stable.html


# from sentence_transformers import SentenceTransformer
# from huggingface_hub import cached_download

# !pip install sentence-transformers==2.2.2 huggingface-hub==0.12.0


# Creating Index with Pinecone Using Code Approach


In [80]:
from dotenv import load_dotenv

load_dotenv()

True

In [81]:
import os
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import SeverlessSpec
import os


#  Initialize Pinecone
pc = Pinecone(api_key="PINECONE_API_KEY"),
        
index = "einseinnmedicalchatbot"

pc.create_index(name="index_name",
                dimension=384,
                metric="cosine",
                spec=SeverlessSpec(),
                cloud="aws", 
                region="us-east-1",
            )

## Let create the Index with Python Codes in Simple Way


In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="pcsk_2jGdBi_RFTHxF67eEHmau3ecngztXbgM8EGVQ1et1SMAJtom2t99yvUUZTWtRYGYAT4Gpd")

index_name = "einseinnmedicalchatbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

# !pip install --upgrade pinecone-client


## CONVERT THE CHUNKS INTO VECTORS EMBEDDINGS

In [82]:
import os
from dotenv import load_dotenv

load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

In [83]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [88]:
#  Embed the text chunks and add them or embeddings to Pinecone Index

from langchain_pinecone import PineconeVectorStore

docs_searching = PineconeVectorStore.from_documents(
     documents=text_chunks,
    index_name=einseinnmedicalchatbot,
    embedding=embeddings,
)

ImportError: cannot import name 'PineconeVectorStore' from 'langchain_pinecone' (/opt/anaconda3/envs/einstienMed/lib/python3.10/site-packages/langchain_pinecone/__init__.py)